In [56]:
import json
from bs4 import BeautifulSoup
from markdownify import markdownify
from dotenv import find_dotenv, load_dotenv
import os
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS

In [52]:
# Load environment variables
load_dotenv(find_dotenv())

True

## Step 1: Create content for vector db

In [13]:
def html_to_markdown(html_string):
    # Parse the HTML string using BeautifulSoup
    soup = BeautifulSoup(html_string, 'html.parser')

    # Convert the parsed HTML to Markdown using markdownify
    markdown_text = markdownify(str(soup))

    return markdown_text

In [3]:
with open('./../data/qa-data.json', 'r') as file:
    data = json.load(file)

In [46]:
def create_content_snippet(question):
    # Get all content pieces in the answer
    content_pieces = list(map(lambda content_piece: [html_to_markdown(paragraph) for paragraph in content_piece.values()], data[0]["answer"]["content"]))
    concat_content = "\n".join([item for row in content_pieces for item in row])

    return f"""Vraag: {question["question"]}
Antwoord:
{html_to_markdown(question["answer"]["introduction"]) if "introduction" in question["answer"].keys() else ""}
{concat_content}

{"Onderwerpen: " + ", ".join(question["answer"]["subjects"]) if "subjects" in question["answer"].keys() else ""}
{"Thema's: " + ", ".join(question["answer"]["themes"]) if "themes" in question["answer"].keys() else ""}
{"Autoriteit: " + question["answer"]["authority"] if "authority" in question["answer"].keys() else ""}"""

In [47]:
for qa in data:
    qa["content"] = create_content_snippet(qa)

## Step 2: Build vector db

In [72]:
# Ready all models
embedding = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [55]:
vectordb_persist_dir = "./../data/faiss_index"

In [58]:
data[0]

{'id': '62bdc113-aee1-4469-bc36-4c7c3fc39a4d',
 'type': 'vraag en antwoord',
 'canonical': 'https://www.rijksoverheid.nl/onderwerpen/leerplicht/vraag-en-antwoord/leerplicht-schoolvakanties',
 'dataurl': 'https://opendata.rijksoverheid.nl/v1/infotypes/faq/62bdc113-aee1-4469-bc36-4c7c3fc39a4d',
 'question': 'Mag ik mijn kind meenemen op vakantie buiten de schoolvakantie?',
 'lastmodified': '2023-05-31T13:52:55.841Z',
 'answer': {'id': '62bdc113-aee1-4469-bc36-4c7c3fc39a4d',
  'type': 'vraag en antwoord',
  'canonical': 'https://www.rijksoverheid.nl/onderwerpen/leerplicht/vraag-en-antwoord/leerplicht-schoolvakanties',
  'dataurl': 'https://opendata.rijksoverheid.nl/v1/infotypes/faq/62bdc113-aee1-4469-bc36-4c7c3fc39a4d',
  'question': 'Mag ik mijn kind meenemen op vakantie buiten de schoolvakantie?',
  'introduction': '<p>U mag uw kind niet meenemen op vakantie buiten de schoolvakanties. U kunt een boete krijgen als u dit wel doet. Als u in de schoolvakanties niet weg kunt door uw werk kun

In [64]:
if not os.path.exists(vectordb_persist_dir):
    factors = [
        Document(
            page_content=qa_item["content"],
            metadata={
                "identifier": qa_item["id"],
                "source": qa_item["canonical"]
            },
        )
        for qa_item in data
    ]

    # Create vector store
    vectordb = FAISS.from_documents(
        documents=factors,
        embedding=embedding,
    )
    vectordb.save_local(vectordb_persist_dir)
else:
    # ChromaDB has been initialised before, recreate instance
    vectordb = FAISS.load_local(vectordb_persist_dir, embedding)

In [73]:
def get_answer_from_llm(query):    
    # Build prompt
    template = """Gedraag je als een helpvolle assistent voor mensen die op zoek zijn naar allerlei antwoorden op vragen die iets te maken hebben met de Rijksoverheid. Beantwoord deze vraag ALLEEN op basis van de gegeven bronnen, niet op basis van eigen kennis. Als je de vraag niet kan beantwoorden, verontschuldig je en zeg dat de webmaster op de hoogte is gebracht van het niet hebben van de gevraagde informatie.
    Bronnen: ```{context}```
    Vraag: ```{question}```"""
    qa_chain_prompt = PromptTemplate.from_template(template)

    # Define search kwargs
    search_kwargs = {"k": 5}

    # Create QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(search_kwargs=search_kwargs),
        return_source_documents=True,
        chain_type_kwargs={"prompt": qa_chain_prompt},
    )

    # Get result
    result = qa_chain({"query": query})

    return (
        result
    )

In [74]:
get_answer_from_llm("Wanneer betaal ik belasting?")

{'query': 'Wanneer betaal ik belasting?',
 'result': 'Op basis van de gegeven bronnen betaalt u belasting in de volgende situaties:\n\n1. U betaalt erfbelasting als u een erfenis krijgt. Hoeveel erfbelasting u moet betalen, hangt af van uw relatie met de overledene en van het bedrag dat u erft.\n\n2. Over energiebelasting betaalt u 21% btw. U betaalt hierdoor btw over de energie en over de energiebelasting.\n\n3. Als u een schenking of gift krijgt, betaalt u schenkbelasting aan de Belastingdienst. Een schenking bestaat meestal uit geld of goederen.\n\nLet op: dit zijn slechts enkele voorbeelden van wanneer u belasting betaalt. Er zijn nog veel meer situaties waarin u belasting moet betalen. Voor meer specifieke informatie kunt u het beste contact opnemen met de Belastingdienst of een belastingadviseur.',
 'source_documents': [Document(page_content="Vraag: Wanneer betaal ik erfbelasting?\nAntwoord:\nU betaalt erfbelasting als u een erfenis krijgt. Hoeveel erfbelasting u moet betalen, ha